# BeautyfulSoup 네이버 영화 리뷰 스크래핑

## 네이버 영화 리뷰 스크래핑

* 네이버 영화: https://movie.naver.com/movie/point/af/list.nhn

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import pandas as pd

def get_movie_reviews(mcode, page_num=10):

  movie_review_df = pd.DataFrame(columns=("Title","Score","Review"))
  url = "https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=189069&target=after" + str(mcode) + "&target=after"
  idx = 0

  for _ in range(0, page_num):
    movie_page = urllib.request.urlopen(url).read()
    movie_page_soup = BeautifulSoup(movie_page, 'html.parser')

    review_list = movie_page_soup.find_all('td', {'class':'title'})
    for review in review_list:
      title = review.find('a', {'class':'movie color_b'}).get_text()
      score = review.find('em').get_text()
      review_text = review.find('a', {'class':'report'}).get('href').split(',')[2]
      movie_review_df.loc[idx]=[title, score, review_text]
      idx +=1
      print("#", end="")
    try:
      url = "https://movie.naver.com" + movie_page_soup.find('a', {'class':'pg_next'}).get('href')
    except:
      break

  return movie_review_df

In [ ]:
movie_review_df = get_movie_reviews(189069,100)
movie_review_df

########################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################

,Title,Score,Review
0,다만 악에서 구하소서,9,'황정민 이정재 박정민 모두 연기 진짜 bb이정재 액션신에서 눈빛 장난 아니었습니다'
1,다만 악에서 구하소서,10,''
2,다만 악에서 구하소서,10,'역시 황정민이랑 이정재는 치트조합이다'
3,다만 악에서 구하소서,10,'영화관에서 3번봄 세번 다 만이천원 주고. 진짜 오랜만에 재밌는 영화였다 가슴 ...
4,다만 악에서 구하소서,7,''
...,...,...,...
995,다만 악에서 구하소서,10,'한국판테이큰임. 지루함없이 정말 재밌게 봤어요.'
996,다만 악에서 구하소서,10,'박정민... 진짜 입덕하고싶은 연기 ... 다른 배우 떠나서 박정민 밖에 보이지...
997,다만 악에서 구하소서,10,'다만
998,다만 악에서 구하소서,10,'이런영화가 영화지현정권 빠는 정치영화는 꺼지길'


## 현재상영작 영화 리뷰 스크래핑

In [ ]:
url = "http://movie.naver.com/movie/point/af/list.nhn"
naver_movie = urllib.request.urlopen(url).read()
soup = BeautifulSoup(naver_movie, 'html.parser')
select = soup.find('select', {'id':'current_movie'})
movies = select.find_all('option')

movies_dict = {}
for movie in movies[1:]:
  movies_dict[movie.get('value')]=movie.get_text()

movie_review_df = pd.DataFrame(columns=("Title","Score","Review"))
for mcode in movies_dict:
  df = get_movie_reviews(mcode,1)
  movie_review_df = pd.concat([movie_review_df, df])

movie_review_df

########################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################

,Title,Score,Review
0,다만 악에서 구하소서,10,'악
1,다만 악에서 구하소서,9,'황정민 이정재 박정민 모두 연기 진짜 bb이정재 액션신에서 눈빛 장난 아니었습니다'
2,다만 악에서 구하소서,10,''
3,다만 악에서 구하소서,10,'역시 황정민이랑 이정재는 치트조합이다'
4,다만 악에서 구하소서,10,'영화관에서 3번봄 세번 다 만이천원 주고. 진짜 오랜만에 재밌는 영화였다 가슴 ...
...,...,...,...
5,다만 악에서 구하소서,7,''
6,다만 악에서 구하소서,10,'스토리 필요없어.. 배우들 표정 눈빛 하나하나가 이미 스토리야'
7,다만 악에서 구하소서,4,'그냥 삼류 아류 영화 다시는 안 볼 영화.개연성도 없고 스토리도 없는 액션만 죽...
8,다만 악에서 구하소서,10,''
